In [1]:
!pip install pandas-datareader

In [66]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [51]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

In [52]:
dataset = data_reader.DataReader('AAPL', data_source="yahoo")
dataset[0:5]

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-02-03,119.089996,117.610001,118.500000,118.650002,51915700.0,109.017654
2015-02-04,120.510002,118.309998,118.500000,119.559998,70149700.0,109.853806
2015-02-05,120.230003,119.250000,120.019997,119.940002,42246200.0,110.637886
2015-02-06,120.250000,118.449997,120.019997,118.930000,43706600.0,109.706200
2015-02-09,119.839996,118.430000,118.550003,119.720001,38889800.0,110.434944


In [53]:
close = dataset['Close']
close[0:5]

Date
2015-02-03    118.650002
2015-02-04    119.559998
2015-02-05    119.940002
2015-02-06    118.930000
2015-02-09    119.720001
Name: Close, dtype: float64

In [55]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close

def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

In [75]:
rr = state_creator(close, 2, 11)

In [76]:
print(rr)

[[0.5        0.5        0.5        0.5        0.5        0.5
  0.5        0.5        0.71299935 0.59387428]]


In [19]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

In [20]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

In [21]:
trader = AI_Trader(window_size)

In [22]:
trader.model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                352       
_________________________________________________________________
dense_5 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

Episode: 1/1000


  0%|                                                                                         | 0/1257 [00:00<?, ?it/s]

AI Trader bought:  $ 118.650002
AI Trader sold:  $ 119.559998  Profit: $ 0.909996
AI Trader bought:  $ 119.940002
AI Trader sold:  $ 118.930000  Profit: - $ 1.010002
AI Trader bought:  $ 128.449997
AI Trader bought:  $ 133.000000
AI Trader bought:  $ 132.169998
AI Trader bought:  $ 130.419998
AI Trader sold:  $ 129.089996  Profit: $ 0.639999
AI Trader sold:  $ 129.360001  Profit: - $ 3.639999
AI Trader bought:  $ 127.139999
AI Trader sold:  $ 124.510002  Profit: - $ 7.659996
AI Trader sold:  $ 122.239998  Profit: - $ 8.180000


  2%|█▋                                                                             | 26/1257 [00:00<00:04, 248.28it/s]

AI Trader sold:  $ 124.449997  Profit: - $ 2.690002
AI Trader bought:  $ 128.470001


...

AI Trader bought:  $ 93.239998


 25%|███████████████████▉                                                           | 317/1257 [14:25<41:29,  2.65s/it]

AI Trader bought:  $ 92.720001


 25%|███████████████████▉                                                           | 318/1257 [14:28<42:14,  2.70s/it]

AI Trader bought:  $ 92.790001
